# CN5 - Champ électrique dans un condensateur

> **L'objectif de ce TD est déterminer l'allure du champ électrique dans un condensateur non infini.**
>
>
> *Vous devez :*
> - rédiger les questions théoriques (T1, T2...) sur une copie que vous rendrez à votre enseignant.
> - exécuter les cellules de code déjà écrites sur ce document, et compléter celles correspondant aux questions "informatique" (Q1, Q2...).
> - enregistrer votre Notebook au format : *CN5_votrenom_votreprenom.ipynb* et le transmettre à votre enseignant via cahier-de-prepa.


## Présentation du problème 
On soumet un condensateur plan infini à une différence de potentiel $U$. L’armature négative du condensateur est au potentiel $-\frac{U}{2}$, et l’armature positive est au potentiel $\frac{U}{2}$. Une densité surfacique de charges $+\sigma$ apparaît sur l’armature positive et une densité $-\sigma$ sur l’électrode négative. 

>**Question T1 :** 
>
> Faire un schéma. Déterminer le champ électrique existant dans le condensateur plan infini. Représenter le champ dans le condensateur. On attend une rédaction soignée pour ce genre de questions classiques.
	
>**Question T2 :** 
>
> En réalité, le condensateur plan n’est pas infini. A quelle condition le résultat précédent est-il toujours valable ?
	
    
    
L’objectif de ce TP est de déterminer numériquement le champ $\vec{E}$ et le potentiel $V$ dans un condensateur non infini, pour le comparer au résultat trouvé précédemment.

>**Question T3 :** 
>
> Quelle est l’équation différentielle vérifiée par $V$ au sein du condensateur ? De quoi a-t-on besoin, en plus de cette équation, pour déterminer $V(M)$ ?

>**Question T4 :** 
>
> Que devient l’équation précédente en 2D (on néglige les variations dans la profondeur du condensateur suivant z :  $V=V(x,y)$ ?

Les modules nécessaires à l'étude seront les suivants :

In [4]:
#Cellule à exécuter

# Modules Python utiles

# Chargement des modules necessaires

import numpy as np  ## pour le calcul numérique
import matplotlib.pyplot as plt   ## pour les tracés

## Géométrie du problème

Nous simulons, en 2D, un condensateur plan mais pas infini. Nous le représentons ci-après.


<img src="https://simon-laurette.github.io/CN5_ChampCondensateur/geo.png" width=400></img>

On appelle “Domaine $\mathcal{D}$ ” l’ensemble des points M de l’espace où l’on va chercher à calculer le potentiel $V(M)$. En réalité, il y a un nombre infini de points $M$ dans le domaine : on ne peut pas calculer $V$ partout.

On va définir un certain nombre de points, suffisamment proches, pour lesquels on va calculer $V$ : on va ainsi réaliser un maillage de l’espace / discrétiser l’espace.

>**Question T5 :** 
>
> Que signifie “suffisamment proches” ? Que risque-t-il de se passer si on définit trop peu de points dans le maillage ? ou trop de points ?

Dans notre cas, on choisit des points $M$ espacés les uns des autres de $a=Δx=Δy=1μm$. Chaque point sera la case d’un tableau défini sur Python. On écrira alors dans cette case la valeur du potentiel. Toute la suite du programme a pour objectif de remplir ce tableau. Commençons par initialiser ce tableau avec les valeurs de potentiels que nous connaissons déjà. Ecrivons pour cela la fonction `remplissage_initial` :

In [6]:
def remplissage_initial(U):
    nl=160
    nc=120
    V=np.zeros((nl,nc))  
    for i in range(60,100):
        V[i,50]=-U/2
        V[i,70]=U/2
    return V

>**Question T6 :** 
>
> Quelle est la taille du domaine $D$ ? Quelle est la distance entre les électrodes ? Quelle est la dimension verticale des électrodes ? 


>**Question Q1 :** 
>
> A l'aide de la fonction `imshow` de `pyplot`, observer la matrice créée par la fonction `remplissage_intial`.


In [7]:
V=remplissage_initial(20)
### A COMPLETER


## Remplissage du tableau des potentiels

Il nous faut maintenant écrire une fonction qui va remplir le tableau en écrivant $V$ dans chaque case inconnue, et donc en chaque point $M$ du domaine. Pour cela, il nous faut résoudre l’équation de Laplace :

$$\frac{\partial^2 V}{\partial x^2}+\frac{\partial^2 V}{\partial y^2}=0$$


Dans notre cas, la fonction $V$ n’est pas définie pour x et y continus, mais discrétisés :

<img src="https://simon-laurette.github.io/CN5_ChampCondensateur/dis.png" width=400></img>

Il va donc falloir approximer $\frac{\partial^2 V}{\partial x^2}$ et $\frac{\partial^2 V}{\partial y^2}$ pour résoudre l’équation précédent dans notre cas.

>**Question T6 :** 
>
> Soit une fonction f. En écrivant la formule de Taylor à l’ordre 2 pour f(x+a) et f(x-a), démontrer que :
$$f''(x)≃\frac{1}{a^2}  [f(x+a)+f(x-a)-2 f(x)]$$

>**Question T7 :** 
>
> En déduire l’expression de $\frac{\partial^2 V}{\partial x^2}$ en fonction de `a`, `V[i,j]`, `V[i,j+1]`,  `V[i,j-1]`. De même, en déduire l’expression de $\frac{\partial^2 V}{\partial y^2}$  en fonction de `a`, `V[i,j]`, `V[i+1,j]`, `V[i-1,j]`.

>**Question T8 :** 
>
> Montrer que l’équation de Laplace devient alors :
`V[i+1,j]+V[i-1,j]+V[i,j+1]+V[i,j-1]-4V[i,j]=0`

Ainsi, lorsque l’équation est résolue, on a en chaque point du maillage (hors armatures) :

```python
V[i,j]=1/4 (V[i+1,j]+V[i-1,j]+V[i,j+1]+V[i,j-1])
```

Ce qui n’est pas le cas dans la matrice initialisée par la fonction `remplissage_initial`. Pour y parvenir, on va procéder par itération avec la méthode suivante :

<img src="https://simon-laurette.github.io/CN5_ChampCondensateur/ite.png" width=400></img>

>**Question Q2 :** 
>
> Ecrire une fonction `iteration` qui prend pour argument le tableau `V` et donne, en sortie, le tableau `V` modifié comme indiqué sur le schéma. Il ne faudra toutefois pas modifier la valeur de $V$ sur les armatures du condensateur. De plus, la formule d’itération ne fonctionne pas sur les frontières du domaine D. Ces frontières étant suffisamment loin du condensateur, on supposera que $V=0$ sur ces frontières : c’est la condition aux limites de notre problème.


In [ ]:
#### A COMPLETER

>**Question Q3 :** 
>
> Afficher le tableau modifié suite à l'application d'une fois la fonction `iteration`.



In [ ]:
### A COMPLETER


>**Question Q4 :** 
>
> Appliquer $N=1000$ fois l’itération précédente et en déduire la matrice `V` solution de l’équation de Laplace.

In [ ]:
### A COMPLETER


>**Question Q5 :** 
>
> Tracer les équipotentielles à l’aide de `plt.contour`.

In [ ]:
### A COMPLETER

 ## Détermination du champ électrique
 
 On va maintenant déterminer le champ électrique $\vec{E}=-\overrightarrow{\text{grad}}(V)$.
 
 >**Question Q6 :** 
Ecrire une fonction `Ex,Ey=champ_elec(V)` qui retournera deux tableaux `Ex` et `Ey`, correpondant aux composantes de $\vec{E}$ suivant $x$ et $y$ en chaque point $M$ du domaine.


In [ ]:
### A COMPLETER

>**Question Q7 :** 
Exécuter et commenter le code suivant pour expliquer le rôle de chaque ligne

In [ ]:
champ_max=(np.max(Ex)**2+np.max(Ey)**2)**0.5
lw=1/champ_max*(Ex**2+Ey**2)**0.5
plt.streamplot(np.linspace(0,nc-1,nc), np.linspace(0,nl-1,nl), Ex, Ey, linewidth=10*lw, density=2)

On doit obtenir une allure de ce type :

<img src="https://simon-laurette.github.io/CN5_ChampCondensateur/all.png" width=400></img>

>**Question Q8 :** 
Commenter en comparaison au résultat obtenu avec le théorème de Gauss pour un condensateur plan infini.